In [123]:
import yfinance as yf
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
import datetime as dt
import isoweek
import math
from sklearn.preprocessing import MinMaxScaler
import curl_cffi
curl_cffi.requests.Session(impersonate="chrome")

In [2]:
def get_data():
    offset = 0
    now = pd.Timestamp.today() - pd.Timedelta(days=3)
    
    # No stock market data on weekends
    if now.day_of_week == 5:
        offset = 1
    elif now.day_of_week == 6:
        offset = 2

    now = now - pd.Timedelta(days=offset)        
    now = now.floor('10 min')
    prev = now - pd.Timedelta(minutes=50)
    data = yf.download('EURUSD=X', start=prev, end=now, interval='5m')
    return data

In [244]:
data = get_data()

/tmp/ipykernel_104346/2587580357.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('EURUSD=X', start=prev, end=now, interval='5m')
[*********************100%***********************]  1 of 1 completed


In [243]:
data.to_csv('~/Workspace/fp/yfinance.csv')

In [5]:
count = 0
high = 0.00000
low = 9.99999
open_price = 0.00000
close_price = 0.00000

clean = pd.DataFrame(columns=['week', 'day', 'hour', 'minute',
                              'open', 'high', 'low', 'close',
                              'momentum', 'avg', 'range', 'ohlc'])

for row in data.itertuples():
    if (row[2] > high):
        high = row[2]
    
    if (row[3] < low):
        low = row[3]

    if (count == 0):
        minute = row[0].minute
        open_price = row[4]

    if (count == 1):
        close_price = row[1]
        hour = row[0].hour
        day = row[0].weekday()
        week = row[0].week
        momentum = open_price = close_price
        avg = ( (low + high) / 2 )
        price_range = high - low
        ohlc = ( (open_price + high + low + close_price) / 4)
        
        new_row = [week, day, hour, minute,
                   open_price, high, low, close_price,
                   momentum, avg, price_range, ohlc]
        clean.loc[len(clean)] = new_row
        
        count = 0
        high = 0
        low = 999
        continue
    
    count += 1

In [6]:
clean

,week,day,hour,minute,open,high,low,close,momentum,avg,range,ohlc
0,38.0,4.0,3.0,20.0,1.179384,1.179523,1.178967,1.179384,1.179384,1.179245,0.000556,1.179315
1,38.0,4.0,3.0,30.0,1.179384,1.179523,1.179245,1.179384,1.179384,1.179384,0.000278,1.179384
2,38.0,4.0,3.0,40.0,1.179106,1.179384,1.179106,1.179106,1.179106,1.179245,0.000278,1.179176
3,38.0,4.0,3.0,50.0,1.178550,1.178967,1.178550,1.178550,1.178550,1.178759,0.000417,1.178655
4,38.0,4.0,4.0,0.0,1.179245,1.179245,1.178689,1.179245,1.179245,1.178967,0.000556,1.179106


In [7]:
year = 2025

In [8]:
def to_normal_vector(value):
    angle = value * math.pi * 2
    x = math.sin(angle)
    y = math.cos(angle)
    x = (x + 1) / 2
    y = (y + 1) / 2
    return(x, y)

In [9]:
def normalize_time(df):
    time = pd.DataFrame(columns=['week_x', 'week_y', 'day_x', 'day_y',
                                 'hour_x', 'hour_y', 'minute_x', 'minute_y'])

    for row in df.itertuples():
        week_ratio = row[1] / isoweek.Week.last_week_of_year(year).week
        week = to_normal_vector(week_ratio)
        day_ratio = row[2] / 7
        day = to_normal_vector(day_ratio)
        hour_ratio = row[3] / 24
        hour = to_normal_vector(hour_ratio)
        minute_ratio = row[4] / 6
        minute = to_normal_vector(minute_ratio)
    
        new_row = [ week[0], week[1], day[0], day[1],
                    hour[0], hour[1], minute[0], minute[1] ]
        time.loc[len(time)] = new_row

    return time

In [10]:
time = normalize_time(clean)

In [11]:
price_columns = ['open', 'high', 'low', 'close',
                'momentum', 'avg', 'range', 'ohlc']
prices = clean[price_columns]
scaler = MinMaxScaler()
scaler.set_output(transform='pandas')
prices = scaler.fit_transform(prices.transpose()) # for one value
prices = prices.transpose()
prices.reset_index(drop=True, inplace=True)

In [12]:
norm_data = pd.concat([time, prices], axis=1)

In [13]:
norm_data

,week_x,week_y,day_x,day_y,hour_x,hour_y,minute_x,minute_y,open,high,low,close,momentum,avg,range,ohlc
0,0.003646,0.439732,0.283058,0.049516,0.853553,0.853553,0.933013,0.25,0.999882,1.0,0.999528,0.999882,0.999882,0.999764,0.0,0.999823
1,0.003646,0.439732,0.283058,0.049516,0.853553,0.853553,0.500000,1.00,0.999882,1.0,0.999764,0.999882,0.999882,0.999882,0.0,0.999882
2,0.003646,0.439732,0.283058,0.049516,0.853553,0.853553,0.066987,0.25,0.999764,1.0,0.999764,0.999764,0.999764,0.999882,0.0,0.999823
3,0.003646,0.439732,0.283058,0.049516,0.853553,0.853553,0.933013,0.25,0.999646,1.0,0.999646,0.999646,0.999646,0.999823,0.0,0.999735
4,0.003646,0.439732,0.283058,0.049516,0.933013,0.750000,0.500000,1.00,1.000000,1.0,0.999528,1.000000,1.000000,0.999764,0.0,0.999882


In [14]:
norm_data.loc[len(norm_data)] = 0

In [16]:
# https://www.geeksforgeeks.org/deep-learning/time-series-forecasting-using-tensorflow/

def create_sequences(data, seq_length=5):
    input_data, target = [], []
    for i in range(len(data) - seq_length):
        input_data.append(data.iloc[i:i+seq_length])
        target.append(data.iloc[i+seq_length])
    return np.array(input_data), np.array(target)

# https://www.geeksforgeeks.org/deep-learning/time-series-forecasting-using-tensorflow/

In [17]:
# step = 50 # predict the 60th minute from 50
X, y = create_sequences(norm_data)

In [18]:
x

""


In [19]:
model = keras.models.load_model('model.keras')

In [21]:
prediction = model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


In [27]:
prediction[0][8:]

array([ 0.79345083,  0.78618103,  0.74010044,  0.41726282,  0.8263778 ,
        0.11771979, -0.03768125,  0.7609472 ], dtype=float32)

In [38]:
pred = prediction[0][8:]

In [48]:
prediction

array([[-0.2145921 ,  0.0750551 , -0.49954632,  0.03436432, -0.011317  ,
        -1.8840845 ,  1.1776438 ,  0.23719978,  0.79345083,  0.78618103,
         0.74010044,  0.41726282,  0.8263778 ,  0.11771979, -0.03768125,
         0.7609472 ]], dtype=float32)

In [53]:
def un_normalize(data):
    data = data.reshape(1, -1)
    scaler.fit(data)
    return scaler.inverse_transform(data)

In [55]:
pred_prices = un_normalize(pred)

In [219]:
pred_prices[0]

array([ 1.5869017 ,  1.5723621 ,  1.4802009 ,  0.83452564,  1.6527556 ,
        0.23543958, -0.0753625 ,  1.5218945 ], dtype=float32)

In [ ]:
time_cols = ['week', 'day', 'hour', 'minute']

In [198]:
last_time = clean[-1:]
last_time = last_time.iloc[0]
last_time = last_time[time_cols]

In [147]:
def float_to_string(num):
    return ('%.2f' % (num,)).rstrip('0').rstrip('.')

In [155]:
def time_to_string(series):
    week = float_to_string(series['week'])
    day = float_to_string(series['day'])
    hour = float_to_string(series['hour'])
    minute = float_to_string(series['minute'])

    stamp = str(year)+' '+week+' '+day+' '+hour+' '+minute
    return dt.datetime.strptime(stamp, '%Y %U %w %H %M')
    

In [157]:
pred_time = time_to_string(last_time)

In [159]:
pred_time.strftime('%Y-%d-%m')

'2025-25-09'

In [229]:
j = clean.to_numpy()
ticker = []
for row in j:
    time_data = pd.Series(row[:4], index=time_cols)
    t = time_to_string(time_data)
    date = t.strftime('%Y-%d-%m')
    open_price = i[4]
    high = i[5]
    low = i[6]
    close = i[7]
    momentum = i[8]
    avg = i[9]
    price_range = i[10]
    ohlc = i[11]

    ticker.append({
        'Date': date,
        'Open': open_price.item(),
        'High': high.item(),
        'Low': low.item(),
        'Close': close.item()
    })

In [231]:
ticker

[{'Date': '2025-25-09',
  'Open': 1.1793843507766724,
  'High': 1.1795234680175781,
  'Low': 1.1789672374725342,
  'Close': 1.1793843507766724},
 {'Date': '2025-25-09',
  'Open': 1.1793843507766724,
  'High': 1.1795234680175781,
  'Low': 1.1789672374725342,
  'Close': 1.1793843507766724},
 {'Date': '2025-25-09',
  'Open': 1.1793843507766724,
  'High': 1.1795234680175781,
  'Low': 1.1789672374725342,
  'Close': 1.1793843507766724},
 {'Date': '2025-25-09',
  'Open': 1.1793843507766724,
  'High': 1.1795234680175781,
  'Low': 1.1789672374725342,
  'Close': 1.1793843507766724},
 {'Date': '2025-25-09',
  'Open': 1.1793843507766724,
  'High': 1.1795234680175781,
  'Low': 1.1789672374725342,
  'Close': 1.1793843507766724}]